# 🗺️ Procesamiento de Rejas - La Florida

Este notebook contiene el flujo completo para procesar los datos de rejas/cierres de pasajes en la comuna de La Florida, Santiago de Chile.

## Contenido

1. **Merge de Archivos**: Combinar datos de diferentes fuentes
2. **Snap to Road**: Ajustar puntos a la red vial de OpenStreetMap
3. **Análisis de Cobertura**: Identificar pasajes sin clasificar

---

### Requisitos

Antes de ejecutar, asegúrate de tener instaladas las siguientes librerías:

```bash
pip install pandas openpyxl folium osmnx scipy shapely
```

## 📦 Importar Librerías

In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np

# Para mapas interactivos
import folium
from folium import plugins

# Para red vial de OpenStreetMap
import osmnx as ox

# Para geometría y búsqueda espacial
from shapely.geometry import Point, LineString
from shapely.strtree import STRtree
from scipy.spatial import cKDTree

# Configuración
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

---

# 1️⃣ Merge de Archivos

En esta sección combinamos los datos de las diferentes fuentes:

- **Rejas_Nicolas.xlsx**: Datos recopilados por Nicolás
- **Rejas_Thomas.xlsx**: Datos recopilados por Thomas
- **Calles_Abiertas.xlsx**: Datos históricos de calles abiertas

### Estructura de datos esperada

Cada archivo debe tener las siguientes columnas (o equivalentes):

| Columna | Descripción |
|---------|-------------|
| `lat` | Latitud del punto |
| `lon` | Longitud del punto |
| `estado` | 0=Cerrada, 1=Abierta, 2=Otro tipo |
| `año` | Año del registro |
| `fuente` | Origen del dato |

### 1.1 Definir rutas de archivos

Modifica estas rutas según la ubicación de tus archivos:

In [ ]:
# === CONFIGURACIÓN DE RUTAS ===
# Modifica estas rutas según tu estructura de carpetas

RUTA_DATOS_ORIGINALES = '../01_datos_originales/'
RUTA_DATOS_PROCESADOS = '../03_datos_procesados/'
RUTA_MAPAS = '../04_mapas_html/'

# Archivos de entrada
ARCHIVO_NICOLAS = RUTA_DATOS_ORIGINALES + 'Rejas_Nicolas.xlsx'
ARCHIVO_THOMAS = RUTA_DATOS_ORIGINALES + 'Rejas_Thomas.xlsx'
ARCHIVO_CALLES = RUTA_DATOS_ORIGINALES + 'Calles_Abiertas.xlsx'

# Archivos de salida
ARCHIVO_COMBINADO = RUTA_DATOS_PROCESADOS + 'Base_Combinada.xlsx'
ARCHIVO_SNAPPED = RUTA_DATOS_PROCESADOS + 'Base_Combinada_Snapped_v2.xlsx'

print("📁 Rutas configuradas")

### 1.2 Función para procesar archivos con diferentes formatos

Esta función detecta automáticamente el formato del archivo y lo estandariza:

In [ ]:
def procesar_archivo(ruta, nombre_fuente):
    """
    Procesa un archivo Excel y lo estandariza al formato común.
    
    Detecta automáticamente si las coordenadas vienen:
    - Como columnas separadas (lat, lon)
    - Como texto combinado ("lat, lon" en una columna)
    
    Parámetros:
    -----------
    ruta : str
        Ruta al archivo Excel
    nombre_fuente : str
        Nombre para identificar el origen de los datos
    
    Retorna:
    --------
    DataFrame con columnas: lat, lon, estado, año, fuente
    """
    
    # Leer archivo
    df = pd.read_excel(ruta)
    print(f"\n📄 Procesando: {ruta}")
    print(f"   Columnas encontradas: {list(df.columns)}")
    print(f"   Filas: {len(df)}")
    
    # Detectar formato de coordenadas
    columnas_lower = [c.lower() for c in df.columns]
    
    # Caso 1: Columnas lat/lon separadas
    if 'lat' in columnas_lower and 'lon' in columnas_lower:
        # Ya tiene el formato correcto
        df_out = df[['lat', 'lon', 'estado', 'año']].copy()
        
    # Caso 2: Coordenadas en columna 'cord' o 'Coordenadas'
    elif 'cord' in columnas_lower or 'coordenadas' in columnas_lower:
        col_coord = 'cord' if 'cord' in columnas_lower else 'Coordenadas'
        # Buscar la columna real (respetando mayúsculas)
        for c in df.columns:
            if c.lower() == col_coord.lower():
                col_coord = c
                break
        
        # Separar coordenadas
        coords = df[col_coord].str.split(',', expand=True)
        df_out = pd.DataFrame({
            'lat': coords[0].astype(float),
            'lon': coords[1].astype(float)
        })
        
        # Buscar columna de estado
        for c in df.columns:
            if 'estado' in c.lower() or 'abierto' in c.lower():
                df_out['estado'] = df[c].astype(int)
                break
        
        # Buscar columna de año
        for c in df.columns:
            if 'año' in c.lower() or 'ano' in c.lower():
                df_out['año'] = df[c].fillna(2024).astype(int)
                break
    
    else:
        raise ValueError(f"No se pudo detectar el formato de coordenadas en {ruta}")
    
    # Agregar fuente
    df_out['fuente'] = nombre_fuente
    
    # Limpiar valores nulos
    df_out = df_out.dropna(subset=['lat', 'lon'])
    
    # Estadísticas
    print(f"   ✅ Procesado: {len(df_out)} filas válidas")
    print(f"   Estados: {df_out['estado'].value_counts().to_dict()}")
    
    return df_out

print("✅ Función de procesamiento definida")

### 1.3 Cargar y combinar archivos

In [ ]:
print("="*70)
print("MERGE DE ARCHIVOS")
print("="*70)

# Procesar cada archivo
df_nicolas = procesar_archivo(ARCHIVO_NICOLAS, 'Nicolas')
df_thomas = procesar_archivo(ARCHIVO_THOMAS, 'Thomas')
df_calles = procesar_archivo(ARCHIVO_CALLES, 'Calles_Abiertas')

# Combinar todos los DataFrames
df_combinado = pd.concat([df_nicolas, df_thomas, df_calles], ignore_index=True)

# Asegurar tipos de datos correctos
df_combinado['estado'] = df_combinado['estado'].astype(int)
df_combinado['año'] = df_combinado['año'].fillna(2024).astype(int)

print("\n" + "="*70)
print("RESUMEN DEL MERGE")
print("="*70)
print(f"\n📊 TOTAL: {len(df_combinado)} puntos")
print(f"\n   Por estado:")
for estado, count in df_combinado['estado'].value_counts().sort_index().items():
    nombre = {0: 'Cerrada', 1: 'Abierta', 2: 'Otro'}[estado]
    print(f"      {estado} ({nombre}): {count}")

print(f"\n   Por fuente:")
for fuente, count in df_combinado['fuente'].value_counts().items():
    print(f"      {fuente}: {count}")

### 1.4 Guardar archivo combinado

In [ ]:
# Guardar el archivo combinado
df_combinado.to_excel(ARCHIVO_COMBINADO, index=False)
print(f"💾 Guardado: {ARCHIVO_COMBINADO}")

# Mostrar primeras filas
print("\n📋 Vista previa de los datos:")
df_combinado.head(10)

---

# 2️⃣ Snap to Road

En esta sección ajustamos las coordenadas de los puntos para que coincidan exactamente con la red vial.

## ¿Por qué es necesario?

Los puntos GPS originales pueden tener pequeños errores de precisión que los ubican "fuera" de la calle. El proceso de **snap to road** proyecta cada punto al segmento de calle más cercano.

```
Antes:                    Después:
                          
    •  (punto original)       
    |                     ────•──── (punto en la calle)
────────── (calle)        
```

## Proceso

1. Descargar la red vial de La Florida desde OpenStreetMap
2. Para cada punto, encontrar el segmento de calle más cercano
3. Proyectar el punto perpendicularmente sobre ese segmento
4. Calcular la distancia de ajuste (para control de calidad)

### 2.1 Descargar red vial de OpenStreetMap

In [ ]:
print("="*70)
print("DESCARGANDO RED VIAL")
print("="*70)

# Definir el área de estudio
LUGAR = "La Florida, Santiago, Chile"

print(f"\n🌐 Descargando red vial de: {LUGAR}")
print("   (Esto puede tomar 1-2 minutos...)\n")

# Descargar el grafo de calles
# network_type='all' incluye todos los tipos de vías
# simplify=True simplifica la geometría para mejor rendimiento
G = ox.graph_from_place(
    LUGAR,
    network_type='all',
    simplify=True
)

print(f"✅ Red vial descargada:")
print(f"   📍 Nodos (intersecciones): {len(G.nodes):,}")
print(f"   🛣️  Aristas (segmentos de calle): {len(G.edges):,}")

### 2.2 Preparar segmentos de calle para búsqueda espacial

In [ ]:
print("\n🔧 Preparando segmentos de calle...")

# Crear una lista de LineStrings (segmentos de calle)
# Cada arista del grafo se convierte en una línea geométrica
segmentos = []

for u, v, data in G.edges(data=True):
    # Obtener coordenadas de los nodos de inicio y fin
    node_u = G.nodes[u]
    node_v = G.nodes[v]
    
    # Crear línea geométrica (segmento)
    linea = LineString([
        (node_u['x'], node_u['y']),  # (lon, lat) del nodo inicial
        (node_v['x'], node_v['y'])   # (lon, lat) del nodo final
    ])
    segmentos.append(linea)

print(f"   ✅ {len(segmentos):,} segmentos preparados")

# Crear índice espacial R-tree para búsqueda eficiente
# Esto permite encontrar el segmento más cercano de forma muy rápida
print("\n🌳 Construyendo índice espacial R-tree...")
arbol_espacial = STRtree(segmentos)
print("   ✅ Índice espacial listo")

### 2.3 Función de Snap to Road

In [ ]:
def snap_punto_a_calle(lat, lon, arbol, segmentos_lista):
    """
    Proyecta un punto a la calle más cercana.
    
    Parámetros:
    -----------
    lat : float
        Latitud del punto original
    lon : float
        Longitud del punto original
    arbol : STRtree
        Índice espacial de los segmentos
    segmentos_lista : list
        Lista de LineStrings (segmentos de calle)
    
    Retorna:
    --------
    tuple: (nueva_lat, nueva_lon, distancia_metros)
    """
    
    # Crear punto geométrico
    punto = Point(lon, lat)  # Shapely usa (x, y) = (lon, lat)
    
    # Encontrar el segmento más cercano usando el índice espacial
    idx_cercano = arbol.nearest(punto)
    segmento_cercano = segmentos_lista[idx_cercano]
    
    # Proyectar el punto sobre el segmento
    # 1. project() calcula la distancia a lo largo del segmento
    # 2. interpolate() obtiene el punto en esa posición
    distancia_en_linea = segmento_cercano.project(punto)
    punto_proyectado = segmento_cercano.interpolate(distancia_en_linea)
    
    # Calcular la distancia de ajuste en metros (aproximación)
    # 1 grado ≈ 111,000 metros en el ecuador
    distancia_grados = punto.distance(punto_proyectado)
    distancia_metros = distancia_grados * 111000
    
    return (punto_proyectado.y, punto_proyectado.x, distancia_metros)

print("✅ Función snap_punto_a_calle definida")

### 2.4 Ejecutar Snap to Road en todos los puntos

In [ ]:
print("="*70)
print("EJECUTANDO SNAP TO ROAD")
print("="*70)

# Listas para almacenar resultados
nuevas_lats = []
nuevas_lons = []
distancias_ajuste = []

total_puntos = len(df_combinado)
print(f"\n🔄 Procesando {total_puntos:,} puntos...\n")

# Procesar cada punto
for idx, row in df_combinado.iterrows():
    # Aplicar snap to road
    nueva_lat, nueva_lon, dist = snap_punto_a_calle(
        row['lat'], row['lon'], 
        arbol_espacial, segmentos
    )
    
    nuevas_lats.append(nueva_lat)
    nuevas_lons.append(nueva_lon)
    distancias_ajuste.append(dist)
    
    # Mostrar progreso cada 1000 puntos
    if (idx + 1) % 1000 == 0:
        progreso = (idx + 1) / total_puntos * 100
        print(f"   Procesados: {idx + 1:,}/{total_puntos:,} ({progreso:.1f}%)")

# Actualizar DataFrame con nuevas coordenadas
df_snapped = df_combinado.copy()
df_snapped['lat'] = nuevas_lats
df_snapped['lon'] = nuevas_lons
df_snapped['dist_ajuste_m'] = distancias_ajuste

print(f"\n✅ Snap to road completado")

### 2.5 Estadísticas del ajuste

In [ ]:
print("="*70)
print("ESTADÍSTICAS DE AJUSTE")
print("="*70)

dist = df_snapped['dist_ajuste_m']

print(f"\n📏 Distancia de ajuste:")
print(f"   • Mínima:  {dist.min():.2f} m")
print(f"   • Media:   {dist.mean():.2f} m")
print(f"   • Mediana: {dist.median():.2f} m")
print(f"   • Máxima:  {dist.max():.2f} m")

print(f"\n📊 Distribución:")
print(f"   • Ajuste < 5m:    {len(dist[dist < 5]):,} puntos ({len(dist[dist < 5])/len(dist)*100:.1f}%)")
print(f"   • Ajuste 5-20m:   {len(dist[(dist >= 5) & (dist < 20)]):,} puntos")
print(f"   • Ajuste 20-50m:  {len(dist[(dist >= 20) & (dist < 50)]):,} puntos")
print(f"   • Ajuste 50-100m: {len(dist[(dist >= 50) & (dist < 100)]):,} puntos")
print(f"   • Ajuste > 100m:  {len(dist[dist >= 100]):,} puntos ⚠️")

if len(dist[dist >= 100]) > 0:
    print(f"\n⚠️  Los puntos con ajuste >100m probablemente están fuera de La Florida")
    print("    o en áreas sin cobertura de OpenStreetMap.")

### 2.6 Guardar datos con snap to road

In [ ]:
# Guardar archivo snapped
df_snapped.to_excel(ARCHIVO_SNAPPED, index=False)
print(f"💾 Guardado: {ARCHIVO_SNAPPED}")

# Vista previa
print("\n📋 Vista previa de los datos con snap:")
df_snapped.head(10)

### 2.7 Generar mapa de puntos snapped

In [ ]:
print("="*70)
print("GENERANDO MAPA DE PUNTOS SNAPPED")
print("="*70)

# Contar por estado
n_cerradas = len(df_snapped[df_snapped['estado'] == 0])
n_abiertas = len(df_snapped[df_snapped['estado'] == 1])
n_otro = len(df_snapped[df_snapped['estado'] == 2])
n_ajuste_grande = len(df_snapped[df_snapped['dist_ajuste_m'] > 100])

# Crear mapa base
centro_lat = df_snapped['lat'].mean()
centro_lon = df_snapped['lon'].mean()

mapa = folium.Map(
    location=[centro_lat, centro_lon],
    zoom_start=14,
    tiles='CartoDB dark_matter'  # Fondo oscuro
)

# Agregar capa de mapa claro como alternativa
folium.TileLayer('OpenStreetMap', name='Mapa Claro').add_to(mapa)

# Definir colores
COLORES = {
    0: '#e74c3c',  # Rojo - Cerradas
    1: '#2ecc71',  # Verde - Abiertas
    2: '#9b59b6'   # Morado - Otro tipo
}

NOMBRES = {
    0: 'CERRADA',
    1: 'ABIERTA',
    2: 'OTRO TIPO'
}

# Crear grupos de capas
grupo_cerradas = folium.FeatureGroup(name=f'Cerradas ({n_cerradas})')
grupo_abiertas = folium.FeatureGroup(name=f'Abiertas ({n_abiertas})')
grupo_otro = folium.FeatureGroup(name=f'Otro tipo ({n_otro})')
grupo_ajuste = folium.FeatureGroup(name=f'Ajuste >100m ({n_ajuste_grande})')

print("\n🔄 Agregando puntos al mapa...")

# Agregar cada punto
for idx, row in df_snapped.iterrows():
    estado = int(row['estado'])
    color = COLORES.get(estado, '#95a5a6')
    
    # Si tiene ajuste grande, va a grupo especial
    if row['dist_ajuste_m'] > 100:
        grupo = grupo_ajuste
        color = '#f39c12'  # Naranja
    else:
        grupo = {0: grupo_cerradas, 1: grupo_abiertas, 2: grupo_otro}.get(estado, grupo_otro)
    
    # Crear popup con información
    popup_text = f"""
    <b>Estado:</b> {NOMBRES.get(estado, 'DESCONOCIDO')}<br>
    <b>Año:</b> {int(row['año'])}<br>
    <b>Fuente:</b> {row['fuente']}<br>
    <b>Ajuste:</b> {row['dist_ajuste_m']:.1f}m
    """
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        popup=popup_text,
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.85,
        weight=3
    ).add_to(grupo)

# Agregar grupos al mapa
grupo_cerradas.add_to(mapa)
grupo_abiertas.add_to(mapa)
grupo_otro.add_to(mapa)
grupo_ajuste.add_to(mapa)

# Agregar leyenda
leyenda_html = f'''
<div style="position: fixed; bottom: 50px; right: 50px; width: 220px;
            background-color: rgba(40,40,40,0.95); border: 2px solid #555; z-index: 9999;
            font-size: 13px; padding: 15px; border-radius: 8px; color: white;">
    <h4 style="margin: 0 0 10px 0;">Rejas La Florida</h4>
    <p><span style="color:#2ecc71;">●</span> Abiertas: {n_abiertas}</p>
    <p><span style="color:#e74c3c;">●</span> Cerradas: {n_cerradas}</p>
    <p><span style="color:#9b59b6;">●</span> Otro tipo: {n_otro}</p>
    <p><span style="color:#f39c12;">●</span> Ajuste >100m: {n_ajuste_grande}</p>
    <hr style="border-color:#555;">
    <p style="font-size:11px; color:#888;">Total: {len(df_snapped)} puntos</p>
</div>
'''
mapa.get_root().html.add_child(folium.Element(leyenda_html))

# Agregar controles
folium.LayerControl().add_to(mapa)
plugins.Fullscreen().add_to(mapa)

# Guardar mapa
archivo_mapa_snapped = RUTA_MAPAS + '1_Mapa_Rejas_Snapped_v2.html'
mapa.save(archivo_mapa_snapped)

print(f"\n💾 Mapa guardado: {archivo_mapa_snapped}")
print("\n📍 Vista previa del mapa:")
mapa

---

# 3️⃣ Análisis de Cobertura: Pasajes Sin Clasificar

En esta sección identificamos los inicios de pasajes que **no tienen clasificación**.

## ¿Qué buscamos?

Queremos encontrar las intersecciones de calles residenciales donde debería haber un dato de reja (cerrada/abierta) pero no lo tenemos registrado.

## Proceso

1. Identificar todos los nodos de la red vial que están en **calles residenciales**
2. Filtrar solo las **intersecciones** (nodos con más de 1 conexión)
3. Comparar con nuestros datos para ver cuáles **ya están clasificados**
4. Los restantes son los **pasajes sin clasificar**

### 3.1 Identificar nodos en calles residenciales

In [ ]:
print("="*70)
print("IDENTIFICANDO PASAJES SIN CLASIFICAR")
print("="*70)

# Tipos de calle que nos interesan (pasajes residenciales)
# - 'residential': Calles residenciales normales
# - 'living_street': Calles de convivencia/pasajes peatonales
TIPOS_RESIDENCIALES = {'residential', 'living_street'}

print(f"\n🏘️  Buscando calles tipo: {TIPOS_RESIDENCIALES}")

# Encontrar todos los nodos que están conectados a calles residenciales
nodos_residenciales = set()

for u, v, data in G.edges(data=True):
    # Obtener el tipo de calle (highway)
    highway = data.get('highway', '')
    
    # El atributo puede ser string o lista
    if isinstance(highway, list):
        tipos = set(highway)
    else:
        tipos = {highway}
    
    # Si es calle residencial, agregar ambos nodos
    if tipos & TIPOS_RESIDENCIALES:
        nodos_residenciales.add(u)
        nodos_residenciales.add(v)

print(f"\n✅ Nodos en calles residenciales: {len(nodos_residenciales):,}")

### 3.2 Filtrar solo intersecciones (excluir finales de calle)

In [ ]:
print("\n🔀 Filtrando intersecciones (excluyendo finales de calle)...")

# Un nodo con grado > 1 es una intersección
# Un nodo con grado = 1 es un final de calle (no nos interesa)

intersecciones = []

for node in nodos_residenciales:
    grado = G.degree(node)  # Número de calles conectadas
    
    if grado > 1:  # Solo intersecciones
        data = G.nodes[node]
        intersecciones.append({
            'lat': data['y'],
            'lon': data['x'],
            'grado': grado,
            'node_id': node
        })

print(f"✅ Intersecciones residenciales: {len(intersecciones):,}")
print(f"   (Excluidos {len(nodos_residenciales) - len(intersecciones):,} finales de calle)")

### 3.3 Identificar cuáles ya tienen clasificación

In [ ]:
print("\n🔍 Comparando con datos clasificados...")

# Crear árbol KD para búsqueda eficiente de puntos cercanos
coords_clasificados = df_snapped[['lat', 'lon']].values
arbol_clasificados = cKDTree(coords_clasificados)

# Umbral de distancia: 30 metros
# Si hay un punto clasificado a menos de 30m, consideramos la intersección como cubierta
UMBRAL_METROS = 30
UMBRAL_GRADOS = UMBRAL_METROS / 111000  # Convertir a grados

# Clasificar cada intersección
intersecciones_faltantes = []
intersecciones_cubiertas = []

for inter in intersecciones:
    # Buscar el punto clasificado más cercano
    distancia, _ = arbol_clasificados.query([inter['lat'], inter['lon']])
    
    if distancia > UMBRAL_GRADOS:
        # No hay punto cercano -> falta clasificar
        intersecciones_faltantes.append(inter)
    else:
        # Ya hay un punto cercano -> cubierta
        intersecciones_cubiertas.append(inter)

print(f"\n📊 Resultado:")
print(f"   ✅ Intersecciones CON clasificación: {len(intersecciones_cubiertas):,}")
print(f"   ❌ Intersecciones SIN clasificación: {len(intersecciones_faltantes):,}")

cobertura = len(intersecciones_cubiertas) / len(intersecciones) * 100
print(f"\n   📈 Cobertura: {cobertura:.1f}%")

### 3.4 Generar mapa de cobertura

In [ ]:
print("="*70)
print("GENERANDO MAPA DE COBERTURA")
print("="*70)

# Contar por estado
n_cerradas = len(df_snapped[df_snapped['estado'] == 0])
n_abiertas = len(df_snapped[df_snapped['estado'] == 1])
n_otro = len(df_snapped[df_snapped['estado'] == 2])
n_faltantes = len(intersecciones_faltantes)

# Crear mapa
mapa_cobertura = folium.Map(
    location=[df_snapped['lat'].mean(), df_snapped['lon'].mean()],
    zoom_start=14,
    tiles='CartoDB dark_matter'
)

folium.TileLayer('OpenStreetMap', name='Mapa Claro').add_to(mapa_cobertura)

# Crear grupos
grupo_cerradas = folium.FeatureGroup(name=f'Cerradas ({n_cerradas})')
grupo_abiertas = folium.FeatureGroup(name=f'Abiertas ({n_abiertas})')
grupo_otro = folium.FeatureGroup(name=f'Otro tipo ({n_otro})')
grupo_faltantes = folium.FeatureGroup(name=f'Sin clasificar ({n_faltantes})')

print("\n🔄 Agregando puntos clasificados...")

# Agregar puntos clasificados
for _, row in df_snapped.iterrows():
    estado = int(row['estado'])
    color = COLORES.get(estado, '#95a5a6')
    grupo = {0: grupo_cerradas, 1: grupo_abiertas, 2: grupo_otro}.get(estado, grupo_otro)
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        popup=f"{NOMBRES.get(estado)} | {int(row['año'])} | {row['fuente']}",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.85,
        weight=3
    ).add_to(grupo)

print("🔄 Agregando intersecciones sin clasificar...")

# Agregar intersecciones faltantes
for inter in intersecciones_faltantes:
    popup_text = f"""
    <b style="color: #95a5a6;">SIN CLASIFICAR</b><br>
    <b>Conexiones:</b> {inter['grado']} calles<br>
    <b>Coord:</b> {inter['lat']:.6f}, {inter['lon']:.6f}
    """
    
    folium.CircleMarker(
        location=[inter['lat'], inter['lon']],
        radius=5,
        popup=popup_text,
        tooltip='Sin clasificar',
        color='#95a5a6',  # Gris
        fill=True,
        fillColor='#95a5a6',
        fillOpacity=0.6,
        weight=2
    ).add_to(grupo_faltantes)

# Agregar grupos al mapa (faltantes primero para que queden debajo)
grupo_faltantes.add_to(mapa_cobertura)
grupo_cerradas.add_to(mapa_cobertura)
grupo_abiertas.add_to(mapa_cobertura)
grupo_otro.add_to(mapa_cobertura)

# Leyenda
leyenda_html = f'''
<div style="position: fixed; bottom: 50px; right: 50px; width: 240px;
            background-color: rgba(40,40,40,0.95); border: 2px solid #555; z-index: 9999;
            font-size: 13px; padding: 15px; border-radius: 8px; color: white;">
    <h4 style="margin: 0 0 10px 0;">Cobertura de Datos</h4>
    <p style="background: rgba(149,165,166,0.3); padding: 5px; border-radius: 4px;">
        <span style="color:#95a5a6;">●</span> <b>Sin clasificar: {n_faltantes}</b>
    </p>
    <hr style="border-color:#555;">
    <p><span style="color:#2ecc71;">●</span> Abiertas: {n_abiertas}</p>
    <p><span style="color:#e74c3c;">●</span> Cerradas: {n_cerradas}</p>
    <p><span style="color:#9b59b6;">●</span> Otro tipo: {n_otro}</p>
    <hr style="border-color:#555;">
    <p style="font-size:11px; color:#888;">
        Cobertura: {cobertura:.1f}%<br>
        Solo calles residenciales
    </p>
</div>
'''
mapa_cobertura.get_root().html.add_child(folium.Element(leyenda_html))

# Controles
folium.LayerControl().add_to(mapa_cobertura)
plugins.Fullscreen().add_to(mapa_cobertura)

# Guardar
archivo_mapa_cobertura = RUTA_MAPAS + '5_Inicios_Faltantes.html'
mapa_cobertura.save(archivo_mapa_cobertura)

print(f"\n💾 Mapa guardado: {archivo_mapa_cobertura}")
print("\n📍 Vista previa del mapa:")
mapa_cobertura

---

# 📋 Resumen Final

In [ ]:
print("="*70)
print("RESUMEN FINAL")
print("="*70)

print(f"\n📊 DATOS PROCESADOS:")
print(f"   • Total de puntos: {len(df_snapped):,}")
print(f"   • Cerradas (estado=0): {n_cerradas:,}")
print(f"   • Abiertas (estado=1): {n_abiertas:,}")
print(f"   • Otro tipo (estado=2): {n_otro:,}")

print(f"\n🗺️  COBERTURA:")
print(f"   • Intersecciones residenciales: {len(intersecciones):,}")
print(f"   • Con clasificación: {len(intersecciones_cubiertas):,}")
print(f"   • Sin clasificación: {len(intersecciones_faltantes):,}")
print(f"   • Cobertura: {cobertura:.1f}%")

print(f"\n💾 ARCHIVOS GENERADOS:")
print(f"   • {ARCHIVO_COMBINADO}")
print(f"   • {ARCHIVO_SNAPPED}")
print(f"   • {archivo_mapa_snapped}")
print(f"   • {archivo_mapa_cobertura}")

print("\n" + "="*70)
print("✅ PROCESO COMPLETADO")
print("="*70)

---

# 📚 Anexo: Conceptos Clave

## Estados de las rejas

| Estado | Descripción | Color en mapa |
|--------|-------------|---------------|
| 0 | Cerrada (con reja) | 🔴 Rojo |
| 1 | Abierta (sin reja) | 🟢 Verde |
| 2 | Otro tipo de cierre | 🟣 Morado |

## Tipos de calle en OpenStreetMap

| Tipo | Descripción |
|------|-------------|
| `residential` | Calle residencial normal |
| `living_street` | Calle de convivencia/pasaje peatonal |
| `service` | Calle de servicio (estacionamientos, accesos) |
| `footway` | Sendero peatonal |
| `primary` | Calle principal/avenida |

## Grado de un nodo

El **grado** de un nodo indica cuántas calles se conectan en ese punto:

- Grado 1: Final de calle (cul-de-sac)
- Grado 2: Continuación de calle o esquina
- Grado 3: Intersección en T
- Grado 4+: Cruce de múltiples calles